In [3]:
# 연습문제 :네이버 영화 리뷰 모으기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import time
import sys
import math
import numpy  
import pandas as pd 
import os
import random

print("=" *80)
print("연습문제 :네이버 영화 리뷰 정보 수집하기 ")
print("=" *80)

query_txt = input('1.크롤링 할 영화의 제목을 입력하세요: ')
query_url = 'https://movie.naver.com'

cnt = int(input('2.크롤링 할 리뷰건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("3.파일을 저장할 폴더명만 쓰세요(예:c:\\py_temp\\):")
if f_dir =='' :
    f_dir='c:\\py_temp\\'
    
print("-" *80)
print("요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~~~")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xlsx'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( )

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get(query_url)
driver.maximize_window()
time.sleep(2)

# 영화 제목으로 검색
element = driver.find_element(By.ID,"ipt_tx_srch")
element.send_keys(query_txt)
driver.find_element(By.XPATH,'//*[@id="jSearchArea"]/div/button').click()
driver.find_element(By.XPATH,'//*[@id="old_content"]/ul[2]/li/dl/dt/a').click()
        
#Step 4 평점 옵션 버튼 클릭
driver.find_element(By.LINK_TEXT,"평점").click()

#Step 5. 현재 페이지 리뷰와 점수 등 내용 수집
# IFRAME 변경
driver.switch_to.frame('pointAfterListIframe')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#현재 총 리뷰 건수를 확인하여 사용자의 요청건수와 비교 후 동기화
result= soup.find('div', class_='score_total').find('strong','total').find('em').get_text()

print("=" *80)
search_cnt = int(result.replace(",",""))

if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)


score2=[]
review2=[]
writer2=[]
wdate2=[]
gogam2=[]
g_gogam2=[]
b_gogam2=[]
dwlist2=[]

count = 0         # 크롤링 건수 카운트 변수
click_count = 1   # 페이지 번호

while ( click_count <= page_cnt )  :
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            score_result = soup.find('div', class_='score_result').find('ul')
            slist = score_result.find_all('li')

            for li in slist:

                count += 1
                
                f = open(ff_name, 'a',encoding='UTF-8')
            
                print("\n")
                print("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count))
                score = li.find('div', class_='star_score').find('em').get_text()
                print("1.별점:","*"*int(score),": ", score)
                score2.append(score)
                f.write("\n")
                f.write("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count) + "\n")
                f.write("1.별점:"+score + "\n")
            
                review = li.find('div', class_='score_reple').find('p').get_text().replace('\n','').strip()
                print("2.리뷰내용:",review)
                f.write("2.리뷰내용:" + review + "\n")
                review2.append(review)
         
                dwlist = li.find('div',class_='score_reple').find_all('em')
                writer = dwlist[0].find('span').get_text()
                print("3.작성자:",writer)
                f.write("3.작성자:" + writer + "\n")
                writer2.append(writer)
                
                wdate = dwlist[1].text
                print('4.작성일자:',wdate)
                f.write("4.작성일자:" + wdate + "\n")
                wdate2.append(wdate)
            
                gogam = li.find('div', class_='btn_area').find_all('strong')
                g_gogam = gogam[0].text
                print('5.공감:',g_gogam)
                f.write("5.공감:" + g_gogam + "\n")
                g_gogam2.append(g_gogam)
                
                b_gogam = gogam[1].text
                print('6.비공감:', b_gogam)
                f.write("6.비공감:" + b_gogam + "\n")
                b_gogam2.append(b_gogam)
                
                if count == cnt :
                    break
            
            time.sleep(random.randrange(1,2))  

            click_count += 1
            
            if click_count > page_cnt :
                break
            else :
                driver.find_element(By.LINK_TEXT,"%s" %click_count).click()

            time.sleep(random.randrange(1,2))  # 3-8 초 사이에 랜덤으로 시간 선택

#Step 11. xls 형태와 csv 형태로 저장하기
naver_movie = pd.DataFrame()
naver_movie['별점(평점)']=score2
naver_movie['리뷰내용']=review2
naver_movie['작성자']=writer2
naver_movie['작성일자']=wdate2
naver_movie['공감횟수']=g_gogam2
naver_movie['비공감횟수']=b_gogam2

# csv 형태로 저장하기
naver_movie.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
naver_movie.to_excel(fx_name ,index=True,engine='openpyxl')

e_time = time.time( )
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

연습문제 :네이버 영화 리뷰 정보 수집하기 
1.크롤링 할 영화의 제목을 입력하세요: 극한직업
2.크롤링 할 리뷰건수는 몇건입니까?: 2000
3.파일을 저장할 폴더명만 쓰세요(예:c:\py_temp\):
--------------------------------------------------------------------------------
요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~~~
전체 검색 결과 건수 : 47201 건
실제 최종 출력 건수 2000


크롤링 할 총 페이지 번호:  200


총 2000 건 중 1 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 일단 진선규님 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 부담없이 진짜 코미디 영화를 본거 같아서 즐거웠어요 조조로 할인받고 봤는데 반값이라 더 재밌어요 밤새다 할거없이 봤는데 최고~~
3.작성자: 신해인(godl****)
4.작성일자: 2019.01.23 11:45
5.공감: 10711
6.비공감: 1036


총 2000 건 중 2 번째 리뷰 데이터를 수집합니다==============
1.별점: ******** :  8
2.리뷰내용: 관람객																																																												류승룡 드디어 심폐소생술 성공!
3.작성자: 조희송(vhif****)
4.작성일자: 2019.01.23 11:37
5.공감: 8943
6.비공감: 899


총 2000 건 중 3 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 관람객																																																												진짜 뻥안치고 ㅈㄴ 웃겼다
3.작성자: 냐미냐미(sc2j****)
4.작성일자: 2019.01.23 12:47
5.공감: 9129
6.비공감: 1307


총 20



총 2000 건 중 41 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 대놓고 약빤 감독이 대놓고 약빤 배우들 데리고 만든 극락의 히로뽕 같은 영화ㅎㅎ걍 웃다가 웃다가 웃다가 웃다가 끝난다ㅎㅎㅎㅎ코믹 영화로 스트레스를 풀수도 있다는 걸 34년만에 깨달은 하루ㅋㅋㅋ
3.작성자: lkwe****
4.작성일자: 2019.01.24 12:51
5.공감: 613
6.비공감: 147


총 2000 건 중 42 번째 리뷰 데이터를 수집합니다==============
1.별점: ******** :  8
2.리뷰내용: 관람객																																																												엄마랑 재밌게 봤어요 시간 후딱가요
3.작성자: 임수현(lim_****)
4.작성일자: 2019.01.23 14:02
5.공감: 654
6.비공감: 191


총 2000 건 중 43 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 진선규배우님의 연기 짱!
3.작성자: chri****
4.작성일자: 2019.01.23 18:14
5.공감: 555
6.비공감: 107


총 2000 건 중 44 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 관람객																																																												생각보다 너무 재미있었습니다 진선규님 코믹연기도 너무 잘하셔서 놀랐네요
3.작성자: 럭키가이(slal****)
4.작성일자: 2019.01.23 14:11
5.공감: 569
6.비공감: 126


총 2000 건 중 45 번째 리뷰 데이터를 수집합니다==============
1.별점: *** :  3
2.리뷰내용: 웃음코드가 나와 안맞는 영화이다. 감독이 여기서는 관객이 웃어야만 해 라고 억지로 연출을 한것같음



총 2000 건 중 91 번째 리뷰 데이터를 수집합니다==============
1.별점: ** :  2
2.리뷰내용: 나만 이상한가??? 뭐가 재밌다는 거야??
3.작성자: 블루오션(clie****)
4.작성일자: 2019.02.10 23:24
5.공감: 173
6.비공감: 53


총 2000 건 중 92 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 이런 한심한 수준의 영화가 천만찍을정도니...참담하다...
3.작성자: toyo****
4.작성일자: 2019.02.06 15:08
5.공감: 193
6.비공감: 74


총 2000 건 중 93 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 진짜 재미없다 억지웃음 유치해서 부끄럽더라
3.작성자: wltl****
4.작성일자: 2019.02.04 08:48
5.공감: 207
6.비공감: 88


총 2000 건 중 94 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 개인적으로 이게 왜 천만넘었는지 이해가 안감... 개봉하기 전부터 별로 끌리지도 않았고 이게 이렇게 흥행할 줄 상상도 못했는데 천만이 넘을줄이야..ㅋㅋ 걍 전형적인 한국 코미디 영화였는데ㅠ 그렇게 요즘 사람들이 웃을...
3.작성자: 노랑(mmoo****)
4.작성일자: 2019.02.08 16:16
5.공감: 171
6.비공감: 57


총 2000 건 중 95 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 관람객																																																												진짜 장르 애매하게 가다 어중이떠중이 되는 경찰물들 많은데 이건 정말 첨부터 끝까지 조오오오오온나 웃깁니다
3.작성자: 김민정(ming****)
4.작성일자: 2019.01.24 15:18
5.공감: 198
6.비공감: 86





총 2000 건 중 141 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 많이 실망 햇습니다.... 내용을 보란건지 연기를 보란건지....
3.작성자: alsg****
4.작성일자: 2019.02.13 10:14
5.공감: 88
6.비공감: 34


총 2000 건 중 142 번째 리뷰 데이터를 수집합니다==============
1.별점: ******** :  8
2.리뷰내용: 관람객																																																												재밌어요ㅋㅋ 형사캐릭터 전부 다 차별화되서 웃김진선규랑 공명ㅋㅋㅋ
3.작성자: 연이(ssod****)
4.작성일자: 2019.01.23 21:21
5.공감: 98
6.비공감: 47


총 2000 건 중 143 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 중간에 나오려다가 돈아까워서 참음...
3.작성자: 우엉(tdlu****)
4.작성일자: 2019.02.12 10:02
5.공감: 83
6.비공감: 33


총 2000 건 중 144 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 첨부터 끝까지 웃음밖에 없는 영화 ㅋ 대박 졸잼
3.작성자: LAMISTUDIO(doub****)
4.작성일자: 2019.01.23 14:40
5.공감: 117
6.비공감: 67


총 2000 건 중 145 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 이게천만? 관객수준... 이러니 한국영화가 발전이있을리가 ㅎㅎㅎ
3.작성자: 인코그니토(hhh8****)
4.작성일자: 2019.02.08 14:46
5.공감: 93
6.비공감: 44


총 2000 건 중 146 번째 리뷰 데이터를 수집합니다==============
1.별점: ****** :  6
2.리뷰내용: 볼만은 했는데그저 그런 우



총 2000 건 중 191 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 이런 영화가 1300만 ? 우리나라 사람들 웃을 일이 그렇게 없나 이런  ... 하 그냥 웃음 코드 안맞으면 노 답
3.작성자: du89****
4.작성일자: 2019.02.12 02:57
5.공감: 55
6.비공감: 28


총 2000 건 중 192 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 이게왜 천만영화일까 억지웃음코드..
3.작성자: cash****
4.작성일자: 2019.11.25 21:39
5.공감: 29
6.비공감: 3


총 2000 건 중 193 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 아무리 생각해도 이게 어케 1600만을 찍었는지 이해가 안됨
3.작성자: ELSA(jwh0****)
4.작성일자: 2019.12.27 18:19
5.공감: 32
6.비공감: 6


총 2000 건 중 194 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 관람객																																																												줄거리 보고 기대했는데 기대만큼 재밌네요 특별한 내용이 있거나하진않지만 오랜만에 재밌는 영화본거같아서 돈이 아깝지 않네요
3.작성자: 히응히응(sksg****)
4.작성일자: 2019.01.23 21:21
5.공감: 32
6.비공감: 6


총 2000 건 중 195 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 9점대 영화가 절대 아님
3.작성자: NatureHolic(ikar****)
4.작성일자: 2020.01.06 07:40
5.공감: 35
6.비공감: 9


총 2000 건 중 196 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1

...



총 2000 건 중 1771 번째 리뷰 데이터를 수집합니다==============
1.별점: ****** :  6
2.리뷰내용: 관람객																																																												그냥 피식거릴정도였지 배꼽잡는다 렌즈빠진다 이건 사기관람평이거나 웃음이 헤프다던가
3.작성자: moa0****
4.작성일자: 2019.02.05 04:12
5.공감: 8
6.비공감: 2


총 2000 건 중 1772 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 아 진짜 웃다가 나왔습니다 ㅋㅋㅋ최고다 진짜
3.작성자: vmt0****
4.작성일자: 2019.01.31 22:33
5.공감: 8
6.비공감: 2


총 2000 건 중 1773 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 관람객																																																												완벽한타인처럼 웃다 나왓어요.연기력들이 워낙 뛰어나신분들이라 잘받습니다.뒷자리 아쩌씨의 웃음소리에 더 웃겻던.ㅋㅋㅋ수원왕갈비치킨 짱
3.작성자: 미미(ahah****)
4.작성일자: 2019.01.28 06:10
5.공감: 8
6.비공감: 2


총 2000 건 중 1774 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: ㅆㅗㅏ....쏴...
3.작성자: 신축빌라강집사(hwan****)
4.작성일자: 2019.01.27 17:18
5.공감: 8
6.비공감: 2


총 2000 건 중 1775 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 류승룡 회식하러 따라가는장면에서 사이다 뿜음
3.작성자: 김퉁이(love****)
4.작성일자: 2019.01.27 00:30
5.공감: 



총 2000 건 중 1821 번째 리뷰 데이터를 수집합니다==============
1.별점: ** :  2
2.리뷰내용: 관람객																																																												벨로  벨로  내안의그놈이  더 잼잇는 내가 이상하건가?
3.작성자: 연가(zkcn****)
4.작성일자: 2019.03.03 17:58
5.공감: 9
6.비공감: 3


총 2000 건 중 1822 번째 리뷰 데이터를 수집합니다==============
1.별점: ****** :  6
2.리뷰내용: 관람객																																																												잼나지만 천만동원까지는 아닌것깉아요
3.작성자: ky97****
4.작성일자: 2019.03.01 17:36
5.공감: 9
6.비공감: 3


총 2000 건 중 1823 번째 리뷰 데이터를 수집합니다==============
1.별점: **** :  4
2.리뷰내용: 관람객																																																												...음 감동없고 스토리없고 그냥 말로 웃기는 내용을 제작한 영화.평소에 개그?좋아하시는분들은 좋아할듯 부모님 데려갔다가 조금 분위기 다운됨 차라리 증인이 훨씬 나을듯
3.작성자: 주지원(rlsa****)
4.작성일자: 2019.02.24 23:43
5.공감: 9
6.비공감: 3


총 2000 건 중 1824 번째 리뷰 데이터를 수집합니다==============
1.별점: ****** :  6
2.리뷰내용: 관람객																																																												생각보다 유치하던데..ㅋㅋㅋ진선규를 위한 영화인듯 ㅋ
3.작성자: 톡톡(rang****)
4.작성일자: 2019.02.24 01:32
5.공감: 9
6.비공감: 3


총 2000 건 중 1825 번째 리뷰 데이터를 수집합니



총 2000 건 중 1861 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 관람객																																																												그냥 계속 웃을 수 있었던 영화!한국영화의 억지 눈물 같은 건 없어서 좋았음다만 뼈 부러지는 소리가 아직도 징그럽..
3.작성자: 미선(good****)
4.작성일자: 2019.01.31 19:41
5.공감: 9
6.비공감: 3


총 2000 건 중 1862 번째 리뷰 데이터를 수집합니다==============
1.별점: ********* :  9
2.리뷰내용: 마지막이 사알짝 아쉬웠고 배테랑 초반이 연상됬으나 초반부터 코믹함이 쭉쭉 이어지면서 재밌었습니다. 배우들 한명 한명 돋보이며 누구 하나 꼽기 어려웠습니다.
3.작성자: 청설모(jkll****)
4.작성일자: 2019.01.24 20:45
5.공감: 9
6.비공감: 3


총 2000 건 중 1863 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 아 잼있어요. 오랫만에 웃다 나왔네요.
3.작성자: 이졍(easy****)
4.작성일자: 2019.01.24 19:05
5.공감: 9
6.비공감: 3


총 2000 건 중 1864 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: ㅋㅋ진짜 너무 재밌고 통쾌하고 유쾌하고 호탕하고 짱짱!!! 남녀노소 다 좋아할만한 영화입니다
3.작성자: 행복전도사(huma****)
4.작성일자: 2019.01.24 18:55
5.공감: 9
6.비공감: 3


총 2000 건 중 1865 번째 리뷰 데이터를 수집합니다==============
1.별점: ******** :  8
2.리뷰내용: 재미있어요~ 영화관에서 남의 시선 의식 안하고 이렇게 웃어본 영화가 처음인듯... 그런데 내용은 좀... 그냥 유



총 2000 건 중 1911 번째 리뷰 데이터를 수집합니다==============
1.별점: ** :  2
2.리뷰내용: 추석특선영화로 보세요.
3.작성자: JoA(jiae****)
4.작성일자: 2019.02.12 21:59
5.공감: 10
6.비공감: 4


총 2000 건 중 1912 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 영화 보고 평점 남기기는 처음입니다. 천만영화다 평점 9점대다 듣고 기대하고 갔는데 완전 노잼... 별점 0점 주고 싶은데 최하가 1점이라 1점 남기고 갑니다.
3.작성자: 야호v(dkwk****)
4.작성일자: 2019.02.12 17:05
5.공감: 10
6.비공감: 4


총 2000 건 중 1913 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 연기는 잘하는건 인정 ..하지만 스토리가 너무 없고  무슨 생각으로 만든지 모르겠다..그냥 아무생각없이 보기에는 좋으나 .. 스토리 개연성이나  전달할려는 목적이나 내용들이 깊은 느낌이 없음 .. 그것도 인정
3.작성자: 조혜림(desi****)
4.작성일자: 2019.02.12 16:55
5.공감: 10
6.비공감: 4


총 2000 건 중 1914 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 코드가 맞지 않으면 재미없을 수 있습니다. 중간에 피식 웃음짓는 신이 3-4차례 나오긴 했습니다. 중간에 밖으로 나오고 싶었습니다. 배우분들 열심히 하시긴 했는데 재미있진 않았습니다.
3.작성자: 597e****
4.작성일자: 2019.02.12 12:52
5.공감: 10
6.비공감: 4


총 2000 건 중 1915 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 나만 당할순없닼ㅋㅋㅋ
3.작성자: 쩌러풀(alsc****)
4.작성일자: 2019.02.12 02:38
5.공감: 1



총 2000 건 중 1961 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 이딴걸 영화라고. 진짜 시간이 아까움 전형적인 한국영화
3.작성자: 비타민(gobl****)
4.작성일자: 2019.03.30 23:15
5.공감: 12
6.비공감: 6


총 2000 건 중 1962 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 이거 재미있다는 사람들 진짜인가..진짜 살면서 본 코미디영화중에 최악이었는데..재미도 없고 감동도 없고..이상한 유머코드에..억지스러운 설정에...개콘보다 재미없는건 처음보네
3.작성자: 라파엘로(drum****)
4.작성일자: 2019.03.30 08:16
5.공감: 12
6.비공감: 6


총 2000 건 중 1963 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 배우들...억지로 웃기려고 애쓰는거...보는 내내 나도 힘들었어요ㅠ돈이 아까워서 끝까지 봤는데, 다시는 그러지말아야겠다 다짐...
3.작성자: gozi****
4.작성일자: 2019.03.25 23:40
5.공감: 12
6.비공감: 6


총 2000 건 중 1964 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 9.2점????   영화보는 수준들 보소....
3.작성자: BURBERRY(kayw****)
4.작성일자: 2019.03.24 17:08
5.공감: 12
6.비공감: 6


총 2000 건 중 1965 번째 리뷰 데이터를 수집합니다==============
1.별점: * :  1
2.리뷰내용: 정말 재미없음.. 영화 해운대도 이게 뭔 천만영화야? 라고 했는데 그것보다 훨씬 더 심각하게 재미없음.. 개그는 유치하고 기승전조폭패싸움.. 어쨋든 천만을 넘겼으니 천만영화이긴하지만 다신 이런영화가 나오지않길..
3.작성자: 쿄쿄(robm****)
4.작성일자: 2019.03.23 03:58
